In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf


2025-01-26 11:10:00.398038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737870000.523463  669889 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737870000.551819  669889 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-26 11:10:00.789987: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [145]:
# Load dataset
data = pd.read_csv('/home2/cslab/Documents/End-to-end-mlops/Mobile-Payment-Fraud-Detect/datasets/PS_20174392719_1491204439457_log.csv',nrows=50000)


In [146]:
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [147]:
data.shape

(50000, 11)

In [148]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   step            50000 non-null  int64  
 1   type            50000 non-null  object 
 2   amount          50000 non-null  float64
 3   nameOrig        50000 non-null  object 
 4   oldbalanceOrg   50000 non-null  float64
 5   newbalanceOrig  50000 non-null  float64
 6   nameDest        50000 non-null  object 
 7   oldbalanceDest  50000 non-null  float64
 8   newbalanceDest  50000 non-null  float64
 9   isFraud         50000 non-null  int64  
 10  isFlaggedFraud  50000 non-null  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 4.2+ MB


In [149]:
data['isFraud'].value_counts()

isFraud
0    49900
1      100
Name: count, dtype: int64


Preprocessing

In [150]:
{column: len(data[column].unique()) for column in data.columns}

{'step': 9,
 'type': 5,
 'amount': 49786,
 'nameOrig': 50000,
 'oldbalanceOrg': 30572,
 'newbalanceOrig': 24328,
 'nameDest': 28499,
 'oldbalanceDest': 25369,
 'newbalanceDest': 9367,
 'isFraud': 2,
 'isFlaggedFraud': 1}

In [151]:
data['type'].unique()

array(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN'],
      dtype=object)

In [152]:
pd.get_dummies(data['type'], prefix='tp')

,tp_CASH_IN,tp_CASH_OUT,tp_DEBIT,tp_PAYMENT,tp_TRANSFER
0,False,False,False,True,False
1,False,False,False,True,False
2,False,False,False,False,True
3,False,True,False,False,False
4,False,False,False,True,False
...,...,...,...,...,...
49995,False,False,False,True,False
49996,False,False,False,True,False
49997,False,False,False,True,False
49998,False,False,False,True,False


In [153]:
data

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
49995,9,PAYMENT,1322.91,C1402545110,1208457.61,1207134.71,M578074960,0.0,0.0,0,0
49996,9,PAYMENT,6900.48,C517372485,1207134.71,1200234.22,M1854114037,0.0,0.0,0,0
49997,9,PAYMENT,17399.12,C1948160352,1200234.22,1182835.10,M1940330634,0.0,0.0,0,0
49998,9,PAYMENT,5485.20,C1586381033,1182835.10,1177349.90,M741912557,0.0,0.0,0,0


In [154]:
def onehot_encode(df, column, prefix):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=prefix)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df


In [155]:
def preprocess_inputs(df):
    df = df.copy()
    
    
    # Drop step, nameOrig, and isFlaggedFraud columns
    df = df.drop(['step', 'nameOrig', 'isFlaggedFraud'], axis=1)
    
    
    # One-hot encode the type column
    df = onehot_encode(df, column='type', prefix='tp')
    
    y = df['isFraud'].copy()
    X = df.drop('isFraud', axis=1).copy()
    
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)
    
    
    # Create a tokenizer and fit it to the customer data
    tokenizer = tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(X_train['nameDest'])
    
    print("Total # of customers:", len(tokenizer.word_index))
    
    
    # Create tokenized customer lists
    customers_train = tokenizer.texts_to_sequences(X_train['nameDest'])
    customers_test = tokenizer.texts_to_sequences(X_test['nameDest'])
    
    
    # Pad sequences
    customers_train = tf.keras.preprocessing.sequence.pad_sequences(customers_train, maxlen=1)
    customers_test = tf.keras.preprocessing.sequence.pad_sequences(customers_test, maxlen=1)
    
    
    # Drop nameDest column
    X_train = X_train.drop('nameDest', axis=1)
    X_test = X_test.drop('nameDest', axis=1)
    
    
    # Scale X with a standard scaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
    
    
    return X_train, X_test, customers_train, customers_test, y_train, y_test

In [156]:
X_train, X_test, customers_train, customers_test, y_train, y_test = preprocess_inputs(data)

Total # of customers: 21017


In [157]:


X_train



,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,tp_CASH_IN,tp_CASH_OUT,tp_DEBIT,tp_PAYMENT,tp_TRANSFER
0,-0.476533,-0.331007,-0.333246,-0.347251,-0.399715,-0.465571,-0.618309,-0.115403,1.135244,-0.324425
1,-0.473382,-0.325069,-0.327862,-0.347251,-0.399715,-0.465571,-0.618309,-0.115403,1.135244,-0.324425
2,0.659963,0.315640,0.469088,2.728245,2.704851,2.147900,-0.618309,-0.115403,-0.880868,-0.324425
3,2.643818,-0.258553,-0.333403,2.149973,3.096149,-0.465571,-0.618309,-0.115403,-0.880868,3.082377
4,-0.476656,-0.322919,-0.325276,-0.347251,-0.399715,-0.465571,-0.618309,-0.115403,1.135244,-0.324425
...,...,...,...,...,...,...,...,...,...,...
34995,-0.282179,-0.332726,-0.333403,-0.312612,-0.378779,-0.465571,1.617313,-0.115403,-0.880868,-0.324425
34996,-0.193468,-0.332726,-0.333403,-0.250166,-0.231651,-0.465571,1.617313,-0.115403,-0.880868,-0.324425
34997,-0.045903,-0.330860,-0.268056,-0.131990,-0.134482,2.147900,-0.618309,-0.115403,-0.880868,-0.324425
34998,-0.474906,-0.332726,-0.333403,-0.071037,-0.237439,-0.465571,1.617313,-0.115403,-0.880868,-0.324425


In [158]:
print(X_train.columns)

Index(['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
       'newbalanceDest', 'tp_CASH_IN', 'tp_CASH_OUT', 'tp_DEBIT', 'tp_PAYMENT',
       'tp_TRANSFER'],
      dtype='object')


In [159]:


y_train



17325    0
13544    0
49844    0
16371    0
13084    0
        ..
7763     0
15377    0
17730    0
28030    0
15725    0
Name: isFraud, Length: 35000, dtype: int64

In [160]:
train_df = pd.concat([X_train, pd.Series(np.squeeze(customers_train), name='customer'), y_train.reset_index(drop=True)], axis=1)

In [161]:
train_df

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,tp_CASH_IN,tp_CASH_OUT,tp_DEBIT,tp_PAYMENT,tp_TRANSFER,customer,isFraud
0,-0.476533,-0.331007,-0.333246,-0.347251,-0.399715,-0.465571,-0.618309,-0.115403,1.135244,-0.324425,3450,0
1,-0.473382,-0.325069,-0.327862,-0.347251,-0.399715,-0.465571,-0.618309,-0.115403,1.135244,-0.324425,3451,0
2,0.659963,0.315640,0.469088,2.728245,2.704851,2.147900,-0.618309,-0.115403,-0.880868,-0.324425,579,0
3,2.643818,-0.258553,-0.333403,2.149973,3.096149,-0.465571,-0.618309,-0.115403,-0.880868,3.082377,146,0
4,-0.476656,-0.322919,-0.325276,-0.347251,-0.399715,-0.465571,-0.618309,-0.115403,1.135244,-0.324425,3452,0
...,...,...,...,...,...,...,...,...,...,...,...,...
34995,-0.282179,-0.332726,-0.333403,-0.312612,-0.378779,-0.465571,1.617313,-0.115403,-0.880868,-0.324425,399,0
34996,-0.193468,-0.332726,-0.333403,-0.250166,-0.231651,-0.465571,1.617313,-0.115403,-0.880868,-0.324425,304,0
34997,-0.045903,-0.330860,-0.268056,-0.131990,-0.134482,2.147900,-0.618309,-0.115403,-0.880868,-0.324425,1149,0
34998,-0.474906,-0.332726,-0.333403,-0.071037,-0.237439,-0.465571,1.617313,-0.115403,-0.880868,-0.324425,685,0


In [162]:
print(X_train.columns)

Index(['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
       'newbalanceDest', 'tp_CASH_IN', 'tp_CASH_OUT', 'tp_DEBIT', 'tp_PAYMENT',
       'tp_TRANSFER'],
      dtype='object')


In [163]:
train_df['isFraud'].value_counts()

isFraud
0    34929
1       71
Name: count, dtype: int64

In [164]:


oversampled_data = train_df.query("isFraud == 1").sample(34858, replace=True, random_state=123)



In [165]:


oversampled_data



,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,tp_CASH_IN,tp_CASH_OUT,tp_DEBIT,tp_PAYMENT,tp_TRANSFER,customer,isFraud
33821,-0.408856,-0.321259,-0.333403,-0.343521,-0.388017,-0.465571,1.617313,-0.115403,-0.880868,-0.324425,1428,1
11627,-0.454176,-0.327894,-0.333403,-0.332356,-0.391365,-0.465571,1.617313,-0.115403,-0.880868,-0.324425,2108,1
28865,0.188942,-0.233736,-0.333403,-0.347251,-0.399715,-0.465571,-0.618309,-0.115403,-0.880868,3.082377,1378,1
23959,2.522990,0.107987,-0.333403,-0.292894,-0.024354,-0.465571,1.617313,-0.115403,-0.880868,-0.324425,55,1
18502,-0.454256,-0.327906,-0.333403,-0.347251,-0.396106,-0.465571,1.617313,-0.115403,-0.880868,-0.324425,358,1
...,...,...,...,...,...,...,...,...,...,...,...,...
20572,-0.451855,-0.327554,-0.333403,-0.347251,-0.296089,-0.465571,1.617313,-0.115403,-0.880868,-0.324425,3181,1
7264,-0.207165,-0.291730,-0.333403,-0.338556,-0.399715,-0.465571,1.617313,-0.115403,-0.880868,-0.324425,7133,1
15636,-0.486670,-0.332651,-0.333403,-0.345580,-0.399715,-0.465571,1.617313,-0.115403,-0.880868,-0.324425,11307,1
14302,3.502961,0.251463,-0.333403,-0.347251,-0.399715,-0.465571,-0.618309,-0.115403,-0.880868,3.082377,1920,1


In [166]:
train_df = pd.concat([train_df, oversampled_data], axis=0).sample(frac=1.0, random_state=123).reset_index(drop=True)


In [167]:
train_df

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,tp_CASH_IN,tp_CASH_OUT,tp_DEBIT,tp_PAYMENT,tp_TRANSFER,customer,isFraud
0,-0.455241,-0.328050,-0.333403,-0.347251,-0.399715,-0.465571,-0.618309,-0.115403,-0.880868,3.082377,13802,1
1,-0.433171,1.673388,1.646507,-0.299754,-0.366017,2.147900,-0.618309,-0.115403,-0.880868,-0.324425,1030,0
2,-0.474980,-0.309417,-0.312245,-0.347251,-0.399715,-0.465571,-0.618309,-0.115403,1.135244,-0.324425,3572,0
3,-0.388713,-0.332726,-0.333403,-0.347251,-0.399715,-0.465571,-0.618309,-0.115403,1.135244,-0.324425,5835,0
4,-0.469634,-0.332726,-0.333403,-0.347251,-0.399715,-0.465571,-0.618309,-0.115403,1.135244,-0.324425,4713,0
...,...,...,...,...,...,...,...,...,...,...,...,...
69853,-0.444370,-0.326458,-0.333403,-0.345862,-0.350505,-0.465571,1.617313,-0.115403,-0.880868,-0.324425,2342,1
69854,-0.455568,-0.328098,-0.333403,-0.347251,-0.399715,-0.465571,-0.618309,-0.115403,-0.880868,3.082377,8999,1
69855,-0.453397,-0.332726,-0.333403,-0.347251,-0.399715,-0.465571,-0.618309,-0.115403,1.135244,-0.324425,12402,0
69856,-0.442919,-0.088634,-0.099816,-0.347251,-0.399715,-0.465571,-0.618309,-0.115403,1.135244,-0.324425,17568,0


In [168]:
print(train_df.columns)

Index(['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
       'newbalanceDest', 'tp_CASH_IN', 'tp_CASH_OUT', 'tp_DEBIT', 'tp_PAYMENT',
       'tp_TRANSFER', 'customer', 'isFraud'],
      dtype='object')


In [169]:
data['isFraud'].value_counts()

isFraud
0    49900
1      100
Name: count, dtype: int64

In [178]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from collections import Counter
import joblib


# Model building
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load dataset
data = pd.read_csv(
    '/home2/cslab/Documents/End-to-end-mlops/Mobile-Payment-Fraud-Detect/datasets/PS_20174392719_1491204439457_log.csv',
    nrows=200000
)

# Function to one-hot encode a column
def onehot_encode(df, column, prefix):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=prefix)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

# Preprocessing function
def preprocess_inputs(df):
    df = df.copy()
    
    # Drop irrelevant columns
    df = df.drop(['step', 'nameOrig', 'isFlaggedFraud'], axis=1)
    
    # One-hot encode the 'type' column
    df = onehot_encode(df, column='type', prefix='tp')
    
    # Separate target and features
    y = df['isFraud'].copy()
    X = df.drop('isFraud', axis=1).copy()
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)
    
    # Tokenizer for 'nameDest' column
    tokenizer = tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(X_train['nameDest'])
    
    print("Total # of customers:", len(tokenizer.word_index))
    
    # Tokenize customer data
    customers_train = tokenizer.texts_to_sequences(X_train['nameDest'])
    customers_test = tokenizer.texts_to_sequences(X_test['nameDest'])
    
    # Pad sequences
    customers_train = tf.keras.preprocessing.sequence.pad_sequences(customers_train, maxlen=1)
    customers_test = tf.keras.preprocessing.sequence.pad_sequences(customers_test, maxlen=1)
    
    # Drop the 'nameDest' column
    X_train = X_train.drop('nameDest', axis=1)
    X_test = X_test.drop('nameDest', axis=1)
    
    # Add customer token sequences as a new feature
    X_train['customer'] = np.squeeze(customers_train)
    X_test['customer'] = np.squeeze(customers_test)
    
    # Standardize features
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
    
    # Save scaler for future use
    joblib.dump(scaler, 'scaler.pkl')
    
    return X_train, X_test, y_train, y_test

# Preprocess the data
X_train, X_test, y_train, y_test = preprocess_inputs(data)

# Display class distribution before SMOTE
print("Class distribution before SMOTE:", Counter(y_train))

# Apply SMOTE for oversampling
smote = SMOTE(random_state=123)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Display class distribution after SMOTE
print("Class distribution after SMOTE:", Counter(y_train_smote))

# Verify shapes of data
print("X_train shape after SMOTE:", X_train_smote.shape)
print("y_train shape after SMOTE:", y_train_smote.shape)

# Save column names and shapes for future use
training_info = {
    'columns': X_train_smote.columns.tolist(),
    'train_shape': X_train_smote.shape,
    'test_shape': X_test.shape
}

print("Training_info",training_info)

joblib.dump(training_info, 'training_info.pkl')


# Build the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_smote.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train_smote, y_train_smote,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    verbose=1
)

# Save the trained model
model.save('fraud_detection_model.h5')

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Classification report
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))


Total # of customers: 71146
Class distribution before SMOTE: Counter({0: 139883, 1: 117})
Class distribution after SMOTE: Counter({0: 139883, 1: 139883})
X_train shape after SMOTE: (279766, 11)
y_train shape after SMOTE: (279766,)
Training_info {'columns': ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'tp_CASH_IN', 'tp_CASH_OUT', 'tp_DEBIT', 'tp_PAYMENT', 'tp_TRANSFER', 'customer'], 'train_shape': (279766, 11), 'test_shape': (60000, 11)}
Epoch 1/100


/home2/cslab/miniconda3/envs/AgenticRag/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8743/8743 ━━━━━━━━━━━━━━━━━━━━ 7s 748us/step - accuracy: 0.9201 - loss: 0.1870 - val_accuracy: 0.9430 - val_loss: 0.1447
Epoch 2/100
8743/8743 ━━━━━━━━━━━━━━━━━━━━ 6s 739us/step - accuracy: 0.9626 - loss: 0.0985 - val_accuracy: 0.9382 - val_loss: 0.1647
Epoch 3/100
8743/8743 ━━━━━━━━━━━━━━━━━━━━ 7s 769us/step - accuracy: 0.9687 - loss: 0.0850 - val_accuracy: 0.9500 - val_loss: 0.1193
Epoch 4/100
8743/8743 ━━━━━━━━━━━━━━━━━━━━ 7s 753us/step - accuracy: 0.9721 - loss: 0.0784 - val_accuracy: 0.9675 - val_loss: 0.0754
Epoch 5/100
8743/8743 ━━━━━━━━━━━━━━━━━━━━ 7s 757us/step - accuracy: 0.9729 - loss: 0.0753 - val_accuracy: 0.9441 - val_loss: 0.1294
Epoch 6/100
8743/8743 ━━━━━━━━━━━━━━━━━━━━ 7s 747us/step - accuracy: 0.9744 - loss: 0.0727 - val_accuracy: 0.9539 - val_loss: 0.1141
Epoch 7/100
8743/8743 ━━━━━━━━━━━━━━━━━━━━ 7s 751us/step - accuracy: 0.9763 - loss: 0.0684 - val_accuracy: 0.9663 - val_loss: 0.0792
Epoch 8/100
8743/8743 ━━━━━━━━━━━━━━━━━━━━ 7s 743us/step - accuracy: 0.9770 - los

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 490us/step - accuracy: 0.9778 - loss: 0.0650
Test Loss: 0.0679, Test Accuracy: 0.9773
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 331us/step
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     59970
           1       0.02      0.77      0.03        30

    accuracy                           0.98     60000
   macro avg       0.51      0.87      0.51     60000
weighted avg       1.00      0.98      0.99     60000



In [1]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import joblib

# Define file paths
model_path = '/home2/cslab/Documents/End-to-end-mlops/Mobile-Payment-Fraud-Detect/model/fraud_detection_model.h5'
scaler_path = '/home2/cslab/Documents/End-to-end-mlops/Mobile-Payment-Fraud-Detect/model/scaler.pkl'
training_info_path = '/home2/cslab/Documents/End-to-end-mlops/Mobile-Payment-Fraud-Detect/model/training_info.pkl'

# Check if the model file exists
if not os.path.exists(model_path):
    print(f"Error: Model file not found at {model_path}")
else:
    # Load the trained model, scaler, and training info
    model = tf.keras.models.load_model(model_path)
    scaler = joblib.load(scaler_path)
    training_info = joblib.load(training_info_path)

    # Function to preprocess a single JSON/dict input
    def preprocess_input(json_input):
        """
        Preprocess a single JSON or dictionary input for prediction.

        Parameters:
            json_input (dict): Input data in JSON or dictionary format.

        Returns:
            np.ndarray: Preprocessed input ready for prediction.
        """
        # Convert to DataFrame for easier processing
        df = pd.DataFrame([json_input])

        # Drop irrelevant columns (if present in the input)
        irrelevant_cols = ['step', 'nameOrig', 'isFlaggedFraud']
        for col in irrelevant_cols:
            if col in df.columns:
                df = df.drop(col, axis=1)

        # One-hot encode the 'type' column
        if 'type' in df.columns:
            df = pd.get_dummies(df, columns=['type'], prefix='tp')

        # Ensure all one-hot encoded columns match training data
        for col in training_info['columns']:
            if col not in df.columns:
                df[col] = 0  # Add missing columns with default value 0
        df = df[training_info['columns']]  # Reorder columns to match training order

        # Handle 'nameDest' column if present
        if 'nameDest' in df.columns:
            tokenizer = tf.keras.preprocessing.text.Tokenizer()
            tokenizer.fit_on_texts(df['nameDest'])
            customers = tokenizer.texts_to_sequences(df['nameDest'])
            customers = tf.keras.preprocessing.sequence.pad_sequences(customers, maxlen=1)
            df['customer'] = np.squeeze(customers)
            df = df.drop('nameDest', axis=1)

        # Standardize features using the saved scaler
        df = pd.DataFrame(scaler.transform(df), columns=training_info['columns'])

        return df.values  # Return as NumPy array for prediction

    # Function to predict fraud likelihood
    def predict_fraud(json_input):
        """
        Predict fraud likelihood for a single input.

        Parameters:
            json_input (dict): Input data in JSON or dictionary format.

        Returns:
            float: Predicted fraud probability.
        """
        preprocessed_input = preprocess_input(json_input)
        prediction = model.predict(preprocessed_input)
        return prediction[0, 0]  # Return the first prediction (fraud probability)

    # Example JSON input
    json_input = {
        "type": "TRANSFER",
        "amount": 10000.0,
        "nameDest": "C123456789",
        "oldbalanceOrg": 20000.0,
        "newbalanceOrig": 10000.0,
        "oldbalanceDest": 10000.0,
        "newbalanceDest": 20000.0
    }

    # Predict fraud probability
    fraud_probability = predict_fraud(json_input)
    print(f"Fraud Probability: {fraud_probability:.4f}")


2025-01-25 01:07:28.483573: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737747448.605454    7057 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737747448.633782    7057 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-25 01:07:28.862133: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Error: Model file not found at /home2/cslab/Documents/End-to-end-mlops/Mobile-Payment-Fraud-Detect/model/fraud_detection_model.h5
